In [ ]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output
from google.cloud import bigquery # SQL table interface on Arcus

# Initialize the client service
client = bigquery.Client()
requestSize = 1500
requestLabel = "2025-02"

tmpTableName = "lab.session_request_2025_02"
tableName = "lab.session_request_2025_02_with_metadata"
outFn = "~/deid_transfer/2025_02_requested_sessions_with_metadata.csv"

In [ ]:
# Step 01: Get the proc_ord_ids that have grades and have not been requested
q = """
with grades_cte as (
  select
    distinct proc_ord_id,
    count(*) as num_graders,
    STRING_AGG(
      distinct concat( grader_name, ": ", cast(grade as STRING)),
      ', '
    ) as grades_string,
    avg(grade) as avg_grade
  from
    lab.grader_table_with_metadata_project_independent
  where
    grader_name not like "Coarse Text Search%"
    and grade >= 0
    and grade <= 2
    and grade_category = "Unique"
    and grade_criteria = "SLIP"
  group by
    proc_ord_id
),
projects_cte as (
  select
    proc_ord_id,
    string_agg(project, ', ') as projects_string
  from
    lab.proc_ord_projects
  group by
    proc_ord_id
)
select
  distinct graded.pat_id,
  graded.proc_ord_id,
  proc.proc_ord_age,
  proc.proc_ord_year,
  proc.proc_ord_desc,
  graded.report_origin_table,
  graded.grade_category,
  grades_cte.grades_string,
  grades_cte.avg_grade,
  projects_cte.projects_string,
  case
    when projects_cte.projects_string like '%Pb%' then 2
    when projects_cte.projects_string like '%SLIP Adolescent%' then 1
    else 0
  end as priority
from
  lab.grader_table_with_metadata_project_independent graded
  join arcus.procedure_order proc on graded.proc_ord_id = proc.proc_ord_id
  join grades_cte on grades_cte.proc_ord_id = graded.proc_ord_id
  join projects_cte on projects_cte.proc_ord_id = graded.proc_ord_id
  join lab.proc_ord_projects proj on graded.proc_ord_id = proj.proc_ord_id
where
  graded.proc_ord_id not in (
    select
      req.proc_ord_id
    from
      lab.requested_sessions_main_with_metadata_project_independent req
  )
  and grades_string not like '%: 0%'
  and graded.report_origin_table = "arcus.procedure_order"
  and grades_cte.num_graders > 1
  and avg_grade > 0
  and avg_grade <= 2
group by
  project,
  graded.pat_id,
  graded.proc_ord_id,
  proc.proc_ord_age,
  proc.proc_ord_year,
  proc.proc_ord_desc,
  graded.report_origin_table,
  grades_cte.grades_string,
  grades_cte.avg_grade,
  projects_cte.projects_string,
  graded.grade_category
order by
  priority desc;
"""

df = client.query(q).to_dataframe()
print(list(df))
print(len(df))

In [ ]:
df[df['priority'] == 2].shape

In [ ]:
# Step 02: create a temporary table with the filtered data from Step 02

typeLookupTable = { 
    "pat_id": "string",
    "proc_ord_id": "string",
    "proc_ord_age": "integer", 
    "proc_ord_year": "integer",
    "proc_ord_desc": "string",
    "report_origin_table": "string",
    "grade_category": "string",
    "avg_grade": "float64"}

cols = typeLookupTable.keys()
df = df[cols]

try:
    job = client.query("drop table "+tmpTableName)
    job.result()
except:
    pass

# Set up the table creation query
q = "create table "+tmpTableName+"( "
for key, value in typeLookupTable.items():
    q += key+" "+value+", "
q = q[:-2]+");"
print(q)

# Run the query to create the table structure
job = client.query(q)
job.result()

# Set up the query to add rows to the table structure for df
q = "INSERT INTO "+tmpTableName+" ("
q += ", ".join(list(typeLookupTable.keys()))
q += ") VALUES "
countAdded = 0

for idx, row in df.head(requestSize).iterrows():  
    q += str(list(row)).replace("[", "(").replace("]", ")")+", "
    countAdded += 1
    
q = q[:-2]+";"

# Add the rows to the table structure
job = client.query(q)
job.result()

print(countAdded, "graded reports added to request")

# print number of rows added and number of rows waiting
print("Number of graded reports added to request:", countAdded)
print("Number of graded reports not requested:", len(df)-countAdded)

In [ ]:
# Step 03: join the temporary table on the patient table to get metadata
# Fields to add from arcus.patient: 
# sex, race, ethnicity, year of birth
# birth_weight_kg, birth_length_cm, gestational_age

# Set up query
q = '''
select
  req.*,
  pat.sex,
  pat.race,
  pat.ethnicity,
  pat.dob_year,
  pat.gestational_age_num,
  cast(pat.birth_weight_kg as float64) as birth_weight_kg,
  cast(pat.birth_length_cm as float64) as birth_length_cm, '''
q += '"'+requestLabel+'" as request_label from '
q += tmpTableName
q += ' req left join arcus.patient pat on req.pat_id = pat.pat_id '
# Run query
metaDf = client.query(q).to_dataframe()
# Look at the resulting dataframe
print(metaDf.shape)
print(list(metaDf))
metaDf.head(20)

In [ ]:
# How many of each avg grade value are in the table?
avg_grades = list(set(metaDf['avg_grade'].values))
for grade in avg_grades:
    print(grade, len(metaDf[metaDf['avg_grade'] == grade]))

In [ ]:
# Step 04: create a new request table and drop the temporary table

# Prepend create table to Step 04 query 
q = "create table "+tableName+" as " + q
print(q)

# Execute query
job = client.query(q)
job.result()

# Check new table exists
testQ = "select * from " +tableName
testDf = client.query(testQ).to_dataframe()
print(testDf.shape)


In [ ]:
# Step 05: Add the requested table to the table of requested scans
q = "insert into lab.requested_sessions_main_with_metadata_project_independent select * from " + tableName +";"
job = client.query(q)
job.result()

In [ ]:
# Step 06: clean up temporary table
dropQ = "drop table " + tmpTableName
job = client.query(dropQ)
job.result()

In [ ]:
q = "select * from "+tableName
df = client.query(q).to_dataframe()
df.to_csv(outFn, index=None)

In [ ]:
df.head(20)